In [1]:
import json
import os

# Handle lighting AI studio
if '/teamspace' in os.getcwd():
    os.chdir('/teamspace/studios/this_studio/llm2vec-da')
    print(os.getcwd())

from transformers import TrainingArguments
from llm2vec_da.data_utils import load_raw_datasets
from llm2vec_da.arguments import DataTrainingArguments, ModelArguments

/teamspace/studios/this_studio/llm2vec-da


In [3]:
with open('configs/mntp/MetaLlama3-sheared.json') as f:
    config = json.load(f)

print(f'Loading data from: {config["dataset_name"]}')
data_args = DataTrainingArguments(dataset_name = config['dataset_name'])
model_args = ModelArguments()
training_args = TrainingArguments("/tmp")

Loading data from: jealk/wiki40b-da-clean


In [4]:
raw_datasets = load_raw_datasets(data_args, model_args)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 109486
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 6173
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6219
    })
})

In [5]:
from transformers import AutoTokenizer
print(f'Loading tokenizer from: {config["model_name_or_path"]}')
tokenizer = AutoTokenizer.from_pretrained(config['model_name_or_path']) 

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Loading tokenizer from: princeton-nlp/Sheared-LLaMA-1.3B


In [6]:
from llm2vec_da.preprocessing import tokenize_datasets, group_texts

tokenized_datasets = tokenize_datasets(raw_datasets, tokenizer, data_args, training_args)

Running tokenizer on every text in dataset:   0%|          | 0/109486 [00:00<?, ? examples/s]

Running tokenizer on every text in dataset:   0%|          | 0/6173 [00:00<?, ? examples/s]

Running tokenizer on every text in dataset:   0%|          | 0/6219 [00:00<?, ? examples/s]

In [7]:
from functools import partial

grouped_datasets = tokenized_datasets.map(partial(group_texts, max_seq_length=512), batched=True)

Map:   0%|          | 0/109486 [00:00<?, ? examples/s]

Map:   0%|          | 0/6173 [00:00<?, ? examples/s]

Map:   0%|          | 0/6219 [00:00<?, ? examples/s]

In [8]:
grouped_datasets.save_to_disk("data/mntp_wiki_dk_512_sheared")

Saving the dataset (0/1 shards):   0%|          | 0/140252 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7895 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8166 [00:00<?, ? examples/s]